In [13]:
# model
import torch
import torch.nn as nn
from torch import optim

# dataset and transformation
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import os

# utils
import numpy as np
from PIL import Image
from tqdm import trange, notebook

In [3]:
# 데이터 폴더 경로
path = 'C:\\Users\\JungHyeona\\Documents\\Project\\AI_deeplearning\\image'

# Data set 정의

In [4]:
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        img = self.images[index]
        label = self.labels[index]
        
        if self.transform is not None:
            img = self.transform(img)
            
        return img, label

# 데이터 불러오기

In [5]:
# 데이터 전처리
transform = transforms.Compose([
     transforms.Resize((64, 64)),
     transforms.ToTensor(),
     transforms.Normalize((0.111850675, 0.11192831, 0.11306531), (0.029490899, 0.0293483175, 0.029255329999999996))
])

In [40]:
images = []
image_name = []

for fol_name in os.listdir(path):  # i = ['Broken', 'Normal']
    folder = os.path.join(path, fol_name)  # ex) path/Normal
    
    if fol_name == "Broken" or fol_name == "Normal":
        for file in os.listdir(folder):
            file = os.path.join(folder, file)  # ex) path/Normal/a.png

            image = Image.open(file)
            #image = transform(image)
            images.append(image)
            
            # 이미지 폴더 이름 저장
            image_name.append(fol_name)

In [41]:
# 라벨 값 변경
for i in range(len(image_name)):
    if image_name[i] == 'Normal':
        image_name[i] = 0
    else:
        image_name[i] = 1

### Dataset

In [43]:
# 데이터셋 인스턴스 생성
dataset = CustomDataset(images, image_name, transform=transform)
# 데이터셋을 train, validation로 분리
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

### DataLoader 생성

In [44]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# BasicBlock

In [14]:
class BasicBlock(nn.Module):
    expansion = 1
    
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

# ResNet

In [25]:
class ResNet(nn.Module):
    
    def __init__(self, num_classes=2):
        super(ResNet, self).__init__()
        
        self.in_channels = 64
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(BasicBlock, 64, 2)
        self.layer2 = self._make_layer(BasicBlock, 128, 3, stride=2)
        self.layer3 = self._make_layer(BasicBlock, 256, 3, stride=2)
        self.layer4 = self._make_layer(BasicBlock, 512, 2, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * BasicBlock.expansion , num_classes)
        
        
    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion)
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x
    
model = ResNet()

In [26]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# 훈련

In [45]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = list(model.parameters()), lr = 0.001)

In [46]:
total_batch = len(train_loader)
print('총 배치의 수 : {}'.format(total_batch))

running_loss = 0.0
correct_predictions = 0
total_predictions = 0

num_epochs = 10
for epoch in notebook.tqdm(range(num_epochs)):
    print("start")
    for images, labels in train_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        images = images#.to(device)
        labels = labels#.to(device)

        # 순전파
        outputs = model(images)
        loss = loss_func(outputs, labels)
        
        # 역전파 및 최적화
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 정확도 계산
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

    epoch_loss = running_loss / len(train_dataset)
    epoch_accuracy = correct_predictions / total_predictions * 100

    print(f'Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_accuracy:.2f}%')

총 배치의 수 : 80


  0%|          | 0/10 [00:00<?, ?it/s]

start
Epoch 1/10 - Loss: 0.0000 - Accuracy: 93.77%
start
Epoch 2/10 - Loss: 0.0000 - Accuracy: 95.91%
start
Epoch 3/10 - Loss: 0.0000 - Accuracy: 96.64%
start
Epoch 4/10 - Loss: 0.0000 - Accuracy: 97.25%
start
Epoch 5/10 - Loss: 0.0000 - Accuracy: 97.73%
start
Epoch 6/10 - Loss: 0.0000 - Accuracy: 97.98%
start
Epoch 7/10 - Loss: 0.0000 - Accuracy: 98.18%
start
Epoch 8/10 - Loss: 0.0000 - Accuracy: 98.18%
start
Epoch 9/10 - Loss: 0.0000 - Accuracy: 98.25%
start
Epoch 10/10 - Loss: 0.0000 - Accuracy: 98.40%


# 모델 평가 함수

In [47]:
def evaluate(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            #inputs, labels = inputs.to(device), labels.to(device)
            inputs, labels = inputs, labels
            outputs = model(inputs)
            
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    accuracy = 100 * correct / total
    
    return accuracy

In [48]:
# 모델 평가
val_accuracy = evaluate(model, val_loader)
print(f'Validation Accuracy: {val_accuracy:.2f}%')

Validation Accuracy: 99.37%


In [49]:
# 모델 저장
model_save = 'C:\\Users\\JungHyeona\\Documents\\Project\\AI_deeplearning\\model_save'
torch.save(model.state_dict(), model_save + '\\B_ResNet18_01.pt')